In [1]:
# -*- coding: utf-8 -*-
"""
@author: ChewingGumKing_OJF
"""

#loads necessary libraries
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
from selenium.webdriver.common.keys import Keys 
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import os
from datetime import datetime
from datetime import date
import openpyxl
from selenium.webdriver.common.action_chains import ActionChains
import os
import warnings
import json
import pandas as pd
import pyperclip as pc
from bs4 import BeautifulSoup as bs
import re
from random import randint
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

warnings.filterwarnings("ignore")


In [2]:
#driver=webdriver.Chrome(r'C:\Users\840 g3\Desktop\chromedriver.exe')
from selenium import webdriver 



chrome_options = Options()

chrome_options.add_argument("--disable-extensions")
# linux only
# chrome_options.add_argument("--headless")
# chrome_options.headless = True 

driver = webdriver.Chrome(r'C:\Users\840 g3\Desktop\chromedriver.exe',options=chrome_options)

In [3]:
url='https://www.beck-seminare.de/suche?query=neue+veranstaltungen&umbraco-campaign=home_neu_suchliste'

driver.get(url)
time.sleep(5)

soup=bs(driver.page_source,'lxml')
cards=soup.find_all('div',{'class':'event__item result__item'})
len(cards)

items=[]
for i in cards:
    tle=' '.join(i.find('p',{'class':'event__title'}).text.split())
    try:
        sam=' '.join(i.find('p',{'class':'event__title--sub'}).text.split())
    except:
        sam=''
    ritle=tle+' '+sam
    link=i.find('a')['href']
    lo=' '.join(i.find('div',{'class':'event__info'}).find_all('p')[0].text.split())
    dat=' '.join(i.find('div',{'class':'event__info'}).find_all('p')[1].text.split()).split('ab')[1].strip()
    
    items.append([ritle,link,lo,dat])

In [7]:
beck=list()
for k in items:
    link=k[1]
    driver.get(link)
    time.sleep(2)
    soupn=bs(driver.page_source,'lxml')
    title=' '.join(soupn.find('h1',{'class':'event__title product__title'}).text.split())
    info=' '.join(soupn.find('div',{'class':'event__info product__info'}).text.split())
    dat=' '.join(soupn.find('div',{'class':'box box__item box__prices box--grey acc mobile open'}).find('div',{'class':'box__head box__title accTrigger'}).find('span',{'class':'d-block'}).text.split())
    if ',' in dat:
        st,en=dat.replace(',','-').split('-')
    else:
        st=en=dat
    pick=[st,en]
    pick=[a.strip() for a in pick]
    spl_dt_obj = [datetime.strptime(v, '%d.%m.%Y') for v in pick]
    date_= [z.strftime('%Y-%m-%d') for z in spl_dt_obj]
    start_date=date_[0]
    end_date=date_[1]
    
    loc=' '.join(soupn.find('div',{'class':'box box__item box__prices box--grey acc mobile open'}).find('div',{'class':'box__head box__title accTrigger'}).find('span',{'class':'d-block pt-2'}).text.split())
    event_info=' '.join(soupn.find('div',{'class':'product__section'}).text.split())
    try:
        tic=' '.join(soupn.find('p',{'class':'price__option'}).find('label').get_text(separator='{').split()).split('{')[-1].strip().replace(',00','').replace('.','')
    except:
        tic=''
    if tic!='':
        if 'Free' in tic or tic=='Free':
            ty='free'
            am=''
            cu=''

        elif re.search('[0-9]+\s?\$?\£?\€?\W?',tic).group():
            ty='paid'
            am=re.search('[0-9]+',tic).group()
            cu=re.search('\$?\£?\€+',tic).group()
        ticket=[{'type': ty,
                 'price':am,
                 'currency': cu}]
        ticket_list=json.dumps(ticket,ensure_ascii=False)
    else:
        ticket_list=''
    
    mai=' '.join(soupn.find('a',{'class':'link email'}).text.split())
    mail_=json.dumps([mai])
    
    sp=soupn.find_all('div',{'class':'col-6 col-md-3 col-lg-2'})
    speak=[]
    for u in sp:
        na=' '.join(u.find('p',{'class','profile__name'}).text.split())
        ti=''
        if ',' in na:
            ti=na.split(',', 1)[1]
            na=na.split(',', 1)[0]
            
        ln=''
        shu={
            'name':na,
            'title':ti,
            'link':ln
        }
        speak.append(shu)
    Speakerlist=json.dumps(speak,ensure_ascii=False)
    
    def gu(luc):
        google_url_for_location="https://www.google.com/search?q="+luc+"&oq="+luc+"&num=1"
        time.sleep(randint(0,3))
        driver.get(google_url_for_location)
        time.sleep(4)
        try:
            google_map_url=driver.find_element_by_id('lu_map').click()
        except:
            try:
                google_map_url=driver.find_element_by_class_name('Xm7sWb').click()
            except:
                google_map_url=driver.find_element_by_class_name('Lx2b0d').click()
        time.sleep(1)
        google_map_url=driver.current_url
#                 print(google_map_url)
        return(google_map_url)
        ######################################
    def get_google_map_url(location):
        try:
            return(gu(location))
        except:
            try:
                return(gu(location))
            except:
                sha=location.split(',')
                try:
                    gu(sha[-3])
                except:
                    try:
                        gu(sha[-2])
                    except:
                        try:
                            gu(sha[-1])
                        except Exception as e:
                            print(location, "; url didn't go through")
                            print(e)
                            return("")


    def countr(locale):
        try:
            google_url_for_location="https://www.google.com/search?q="+locale+"&oq="+locale+"&num=1"
            time.sleep(randint(0,3))
            driver.get(google_url_for_location)
            time.sleep(4)
            try:
                google_map_url=driver.find_element_by_id('lu_map').click()
            except:
                try:
                    google_map_url=driver.find_element_by_class_name('Xm7sWb').click()
                except:
                    try:
                        google_map_url=driver.find_element_by_class_name('Lx2b0d').click()
                    except:
                        google_url_for_location="https://www.google.com/search?q="+locale+"&oq="+locale+"&num=1"
                        time.sleep(randint(0,3))
                        driver.get(google_url_for_location)
                        time.sleep(4)
                        google_map_url=driver.find_element_by_id('lu_map').click()
            time.sleep(1)
            try:
                country=driver.find_element_by_class_name('x3AX1-LfntMc-header-title-VdSJob').text
            except:
                country=driver.find_element_by_class_name('bwoZTb').text
#                 print(country)
            return(country)
        except Exception as e:
            print(locale, ";country didn't go through")
            print(e)
            return("")

    time_=''
    org_name='Die BeckAkademie Seminare'
    org_web='https://www.beck-seminare.de/'
    org_pro='Die BeckAkademie Seminare zählt deutschlandweit zu den renommiertesten Seminaranbietern im Bereich Recht und bietet Ihnen maßgeschneiderte Aus- und Fortbildung auf höchstem Niveau.'
    logo=''    
    sponsor=''
    agendalist=''
    type_=''
    category=''
#     Speakerlist='' 
#     ticket_list=''
    event_web=link

    if 'nline' in loc or 'ff Campus' in loc or loc=='' or loc==' ' or loc=='-' or loc=='Virtual' or 'nspecified' in loc or 'irtual' in loc:
        on_off='1'
        city=''
        venue=''
        country=''
        googlePlaceUrl=''
    else:
        on_off='0'
        city=loc#.split()[-3]
        country=countr(city)
        venue=loc
        googlePlaceUrl=get_google_map_url(venue)
    
            
    
    beck.append([link,title,start_date,end_date,time_,event_info,ticket_list,
                 org_pro,org_name,org_web,logo,sponsor,agendalist,
                 type_,category,city,country,venue,event_web,googlePlaceUrl,mail_,
                 Speakerlist,on_off])


Köln ;country didn't go through
Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="lu_map"]"}
  (Session info: chrome=103.0.5060.114)
Stacktrace:
Backtrace:
	Ordinal0 [0x00FC6463+2188387]
	Ordinal0 [0x00F5E461+1762401]
	Ordinal0 [0x00E73D78+802168]
	Ordinal0 [0x00EA1880+989312]
	Ordinal0 [0x00EA1B1B+989979]
	Ordinal0 [0x00ECE912+1173778]
	Ordinal0 [0x00EBC824+1099812]
	Ordinal0 [0x00ECCC22+1166370]
	Ordinal0 [0x00EBC5F6+1099254]
	Ordinal0 [0x00E96BE0+945120]
	Ordinal0 [0x00E97AD6+948950]
	GetHandleVerifier [0x012671F2+2712546]
	GetHandleVerifier [0x0125886D+2652765]
	GetHandleVerifier [0x0105002A+520730]
	GetHandleVerifier [0x0104EE06+516086]
	Ordinal0 [0x00F6468B+1787531]
	Ordinal0 [0x00F68E88+1805960]
	Ordinal0 [0x00F68F75+1806197]
	Ordinal0 [0x00F71DF1+1842673]
	BaseThreadInitThunk [0x7602FA29+25]
	RtlGetAppContainerNamedObjectPath [0x77077A9E+286]
	RtlGetAppContainerNamedObjectPath [0x77077A6E+238]

Köln ; url didn't go through
Message: no

In [8]:
len(beck)

37

In [9]:
beck

[['https://www.beck-seminare.de/eingruppierung-nach-tvoed-tv-l-fuer-einsteiger/product/17429636',
  'Eingruppierung nach TVöD / TV-L für Einsteiger',
  '2022-11-29',
  '2022-11-29',
  '',
  'Topseller Neu Live-Webinar Eingruppierung nach TVöD / TV-L für Einsteiger Grundlegende und aktuelle Rechtsfragen Seminar: 6,5 Zeitstunden / Live-Webinar: 5 Zeitstunden nach § 15 FAO Seminar: 09:00 - 17:00 Uhr / Live-Webinar: 1. Block: 09:30 - 12:00 Uhr; 2. Block: 12:30 - 15:00 Uhr Teilen Programm herunterladen',
  '[{"type": "paid", "price": "449", "currency": "€"}]',
  'Die BeckAkademie Seminare zählt deutschlandweit zu den renommiertesten Seminaranbietern im Bereich Recht und bietet Ihnen maßgeschneiderte Aus- und Fortbildung auf höchstem Niveau.',
  'Die BeckAkademie Seminare',
  'https://www.beck-seminare.de/',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  'https://www.beck-seminare.de/eingruppierung-nach-tvoed-tv-l-fuer-einsteiger/product/17429636',
  '',
  '["seminare@beck.de"]',
  '[{"na

In [10]:
beck_dict=beck


beck_df= pd.DataFrame(columns=['scrappedUrl','eventname','startdate','enddate','timing','eventinfo','ticketlist','orgProfile','orgName','orgWeb','logo','sponsor','agendalist','type','category','city','country','venue','event_website','googlePlaceUrl','ContactMail','Speakerlist','online_event'],data=beck_dict)
beck_df.to_csv("beck.tsv", sep = '\t',index=False)
print('Dusted')

Dusted


In [ ]:
# driver.switch_to.window(driver.window_handles[0])

In [7]:
# s=1
# for k in items:
#     #time.sleep(1)
#     driver.execute_script("window.open('');")
#     driver.switch_to.window(driver.window_handles[s])
#     driver.get(k[1])
#     s+=1
# print(s+1)

39


In [ ]:
# s=1
# for k in cards:
#     #time.sleep(1)
#     driver.execute_script("window.open('');")
#     driver.switch_to.window(driver.window_handles[s])
#     driver.get(k[0])
#     s+=1
# print(s+1)

In [ ]:
# for k in range(1,31):
#     time.sleep(1)
#     driver.switch_to.window(driver.window_handles[k])
